In [1]:
import math
import collections
import pickle
import random
import scipy
import json

from matplotlib import pyplot as plt

import GPy
import numpy as np
import pandas as pd
from tqdm import tqdm


### 1. Perform conversions from volumes and carbon source name to design space

In [2]:
root_path = '/Users/harini/Dropbox (MIT)/Harini Narayanan’s files/1_Home/Biotech/MediaOptimization/BO_MediaOptimization/Molecules/'
carbon_source_filepath = '/Users/harini/Dropbox (MIT)/Harini Narayanan’s files/1_Home/Biotech/MediaOptimization/BO_MediaOptimization/Molecules/'


In [3]:
Stock_solid = pd.read_excel(carbon_source_filepath  + 'CarbonSourceInfo.xlsx', 'Stocks_solid')
Stock_liquid = pd.read_excel(carbon_source_filepath  + 'CarbonSourceInfo.xlsx', 'Stocks_liquid')

CARBON NAME

In [4]:
Carbon_Names = Stock_solid['Carbon Source'].values.tolist()
Carbon_Names.append(Stock_liquid['Carbon Source'][1])
Carbon_Names.append(Stock_liquid['Carbon Source'][2])

UPPER BOUNDS

In [5]:
Carbon_Ub = [50]*17 #[g/L]
Carbon_Ub.append(10) # Glycerol [mL/L]
Carbon_Ub.append(50) #Ethanol [mL/L]

OG_Gly_Ub = 100 #[mL/L]
Met_Ub = 100 #[mL/L]

STOCK CONCENTRATIONS

In [6]:
Stock_Conc = Stock_solid['g/mL'].values.tolist()
Stock_Conc.append(Stock_liquid['mL/mL'][1].tolist())
Stock_Conc.append(Stock_liquid['mL/mL'][2].tolist())

OG_Stock_Conc = Stock_liquid['mL/mL'][0].tolist()

In [7]:
Exp_Used = pd.read_csv('Actual_Round5_3mL.csv')
initN = Exp_Used.shape[0]
print(initN)
Exp_Used_CT = Exp_Used["Carbon_Type"].values
Exp_Used_CC = Exp_Used["Conc_Carbon [uL]"].values
Exp_Used_Gly = Exp_Used["Gly_OG [uL]"].values
Exp_Used_Met = Exp_Used["Met_Prod [uL]"].values

11


In [8]:
Exp_Used

,Unnamed: 0,Carbon_Type,Conc_Carbon [uL],Gly_OG [uL],Met_Prod [uL]
0,0,Galactose,579.484902,112.50,45.00
1,1,Galactose,719.360568,11.25,111.00
2,2,Xylose,71.603623,15.00,42.00
3,3,Galactose,919.182948,180.00,126.00
4,4,Xylose,179.009057,3.75,18.00
5,5,Galactose,419.626998,0.00,63.00
6,6,D-Arabinose,63.771977,11.25,126.00
7,7,Galactose,159.857904,41.25,81.00
8,8,Galactose,209.813499,7.50,33.00
9,9,Galactose,79.928952,7.50,12.00


In [9]:
Des_recon_CT = []
Des_recon_CC = []
Des_recon_Gly = []
Des_recon_Met = []

for i in range(initN):
    index = Carbon_Names.index(Exp_Used_CT[i])
    Des_recon_CT.append(index)
    temp_factor = Carbon_Ub[index]/ Stock_Conc[index]
    cc = Exp_Used_CC[i]/ (temp_factor * 3)
    Des_recon_CC.append(cc)
    gly = Exp_Used_Gly[i] * OG_Stock_Conc/(OG_Gly_Ub * 3)
    Des_recon_Gly.append(gly)
    met = Exp_Used_Met[i]/( Met_Ub * 3)
    Des_recon_Met.append(met)

In [10]:
Design_actual = {'Carbon_Type': Des_recon_CT,
               'Conc_Carbon': Des_recon_CC,
               'Gly_OG': Des_recon_Gly,
               'Met_Prod': Des_recon_Met}

In [11]:
pd.DataFrame(Design_actual).to_csv('Reconstructed_Round5.csv')

In [ ]:
Carbon_Names[18]